# Introduction
This tutorial illustrates how to (1) construct simple circuit in OpenDSS , (2) visualize and export simulation results. <span style='color:red'> Remember to change the working directory in below before you run the script</span>.

In [1]:
import os
os.chdir('C:\\Users\\anson\\Desktop\\OpenDss Tutorial\\T1') #remember to change the directory first

### (1) Construct a simple circuit in OpenDSS
The circuit example is composed of the followings:
- 3.3kV ac voltage source
- 1km transmission line ($Z_{line}$) with resistance 10 ohm/km, connect the voltage source and the two loads below
- Load 1 ($Z_{1}$) with rated power of 30kVA and power factor of 0.5 (leading)
- Load 2 ($Z_{2}$) with rated power of 15kW and power factor of 0.707 (lagging) 
<img src='T1_simple_circuit.png' style='width:500px;height:250px'/> 

The parameters below for defining loads and power sources are often used and therefore important to be known
- basekV: base line-to-line voltage of voltage source
- pu: per unit voltage of the voltage source. (assumed balanced for all phases)
- kV: nominal voltage for loads
- kW/ kVar/ kVa: nominal active/reactive/apparent power of a load
- pf: power factor of a load, negative for leading load 
- phases: No. of phases which the load or power source is operating

Generally, baseKV, kV and phases must be specified for all power source and loads.
<span style='color:green'> Below is the script of openDSS highlighted in green colour</span>. All the comments are followered by either ' // ' or ' ! ' signs
whereas '~' means to follow from the previous line

<span style='color:green'>
//simple one Voltage source two loads circuit

Clear

!Define name of new circuit and voltage source \
New circuit.single_phase_double_loads \
~ bus1=SourceBus phases=1 basekV=3.3 pu=1

!Line (R=resistance, X=reactance, C=capacitance, 1 and 0 correspond to positve and zero sequence respectively) \
New line.V-Load phases=1 bus1=SourceBus bus2=LoadBus unit=km length=1 R1=10 X1=0 c1=0 

!Loads  (negative power factor pf=leading) \
New load.load1 bus1=LoadBus phases=1 kVA=30 pf=-0.5 kV=3.3 \
New load.load2 bus1=LoadBus phases=1 kW=15 pf=0.707 kV=3.3

Solve
</span>     

### (2) Visualise parameters and export simulation results



In this section, visulaisation of simulation results using OpenDSSDirect module will be gone through. It is useful to see if the simulation parameters are defined as expected. To illustrate this, we will try to modify the loads in the example. Firstly, let's run the dss file. 

In [2]:
import opendssdirect as dss
dss.run_command('Redirect single_phase_double_loads.dss')

''

To visualise the parameters, we can use the openDSSDirect module, opendssdirect.utils.X_to_dataframe, where X can be loads,voltage sources, etc. If we want to look at the unshown column, we can access the columns as the way we manipulate dataframe. The detail description of the parameter can be found using the search bars of the websites below.
- https://dss-extensions.org/OpenDSSDirect.py/opendssdirect.html
- https://dss-extensions.org/OpenDSSDirect.jl/latest/

In [3]:
dss.utils.loads_to_dataframe()

,CVRCurve,CVRvars,CVRwatts,CFactor,Class,Growth,IsDelta,Model,Name,NumCust,...,kV,kW,kVABase,kvar,kWh,kWhDays,puSeriesRL,XfkVA,Phases,AllocationFactor
load1,,2,1,4,1,,False,1,load1,1,...,3.3,15,30,-25.9808,0,30,50,0,1,0.5
load2,,2,1,4,1,,False,1,load2,1,...,3.3,15,21.2164,15.0045,0,30,50,0,1,0.5


### Add loads
Now, we will try to add a load connected to the load bus. We can check so by using the opendssdirect module. 

In [4]:
dss.run_command('New load.load3 bus1=LoadBus phases=1 kVA=60 pf=-0.5 kV=3.3') #add loads
dss.utils.loads_to_dataframe() #visualise the parameters

,CVRCurve,CVRvars,CVRwatts,CFactor,Class,Growth,IsDelta,Model,Name,NumCust,...,kV,kW,kVABase,kvar,kWh,kWhDays,puSeriesRL,XfkVA,Phases,AllocationFactor
load1,,2,1,4,1,,False,1,load1,1,...,3.3,15,30,-25.9808,0,30,50,0,1,0.5
load2,,2,1,4,1,,False,1,load2,1,...,3.3,15,21.2164,15.0045,0,30,50,0,1,0.5
load3,,2,1,4,1,,False,1,load3,1,...,3.3,30,60,-51.9615,0,30,50,0,1,0.5


### Remove (disable) loads
Similarly, we can deactivate the loads and verify so. The dataframe below shows that all the loads are exactly the same as before.

In [5]:
dss.run_command('disable load.load3 ') 
dss.utils.loads_to_dataframe() 

,CVRCurve,CVRvars,CVRwatts,CFactor,Class,Growth,IsDelta,Model,Name,NumCust,...,kV,kW,kVABase,kvar,kWh,kWhDays,puSeriesRL,XfkVA,Phases,AllocationFactor
load1,,2,1,4,1,,False,1,load1,1,...,3.3,15,30,-25.9808,0,30,50,0,1,0.5
load2,,2,1,4,1,,False,1,load2,1,...,3.3,15,21.2164,15.0045,0,30,50,0,1,0.5


### Modifying loads
Lastly, we will demonstrate how to modify certain parameters of a load. It is particularly useful if we want to study the effect of changing certain parameters of a network.

In [6]:
dss.run_command('load.load1.kW=10') #decrease the rated power of load1 from 15kW to 10kW
dss.utils.loads_to_dataframe()

,CVRCurve,CVRvars,CVRwatts,CFactor,Class,Growth,IsDelta,Model,Name,NumCust,...,kV,kW,kVABase,kvar,kWh,kWhDays,puSeriesRL,XfkVA,Phases,AllocationFactor
load1,,2,1,4,1,,False,1,load1,1,...,3.3,10,20,-17.3205,0,30,50,0,1,0.5
load2,,2,1,4,1,,False,1,load2,1,...,3.3,15,21.2164,15.0045,0,30,50,0,1,0.5


### Export results

The Simulation results can be obtained using the export and show command as shown Below. 'Export' function will output a csv file whereas 'Show' function will output a text file. Note that you have to add the command <span style='color:red'> dss.run_command('Solve') </span> if you want to get the simulation result with modified parameters on loads. 

Lastly, Jupyter will cease if the text editor(notepad) pop up. Therfore, <span style='color:red'>dss.Basic.AllowEditor(False)</span> command is needed to prohibit the opening of notepad. However, you can open the txt. file manually to look at the summmary of simulation result.

In [7]:
dss.run_command('Export Powers')
dss.run_command('Export Currents')

'C:\\Users\\anson\\Desktop\\OpenDss Tutorial\\T1\\single_phase_double_loads_EXP_CURRENTS.CSV'

In [8]:
dss.Basic.AllowEditor(False) #prohibit the opening of notepad by OpenDss
dss.run_command('Show Powers kVA Elem')
dss.run_command('Show Currents Elem')

''